In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
import pymysql
pymysql.install_as_MySQLdb()

## National Footprint Accounts 2018 from kaggle.com

In [2]:
NFA_df=pd.read_csv("NFA_2018.csv")
print(NFA_df.shape)
print(NFA_df.columns)
NFA_df.head()

(87020, 15)
Index(['country', 'ISO alpha-3 code', 'UN_region', 'UN_subregion', 'year',
       'record', 'crop_land', 'grazing_land', 'forest_land', 'fishing_ground',
       'built_up_land', 'carbon', 'total', 'Percapita GDP (2010 USD)',
       'population'],
      dtype='object')


,country,ISO alpha-3 code,UN_region,UN_subregion,year,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,Percapita GDP (2010 USD),population
0,Armenia,ARM,Asia,Western Asia,1992,BiocapPerCap,1.611286e-01,0.135023,0.083836,0.013718,0.033669,0.000000e+00,4.273741e-01,949.033,3449000
1,Armenia,ARM,Asia,Western Asia,1992,BiocapTotGHA,5.558130e+05,465763.337400,289190.662300,47320.224590,116139.598200,0.000000e+00,1.474227e+06,949.033,3449000
2,Armenia,ARM,Asia,Western Asia,1992,EFConsPerCap,3.909225e-01,0.189137,0.000001,0.004138,0.033669,1.112225e+00,1.730092e+00,949.033,3449000
3,Armenia,ARM,Asia,Western Asia,1992,EFConsTotGHA,1.348487e+06,652429.066600,4.327841,14272.803690,116139.598200,3.836620e+06,5.967954e+06,949.033,3449000
4,Armenia,ARM,Asia,Western Asia,1992,EFExportsPerCap,1.124910e-03,0.002283,0.000000,0.000438,0.000000,4.819043e-02,5.203676e-02,949.033,3449000


In [3]:
countries_df=NFA_df[["ISO alpha-3 code","country"]].drop_duplicates(keep="first")

# find missing values in "code" column
print(len(countries_df["ISO alpha-3 code"].unique()) == len(countries_df["country"].unique()))
drop_list=countries_df[countries_df["ISO alpha-3 code"].isna()]["country"]
print(drop_list)
countries_df=countries_df[~(countries_df["country"].isin(drop_list))]

False
23020      Ethiopia PDR
69780    Sudan (former)
85700             World
Name: country, dtype: object


In [4]:
NFA_2_df=NFA_df[~(NFA_df["country"].isin(drop_list))]

In [5]:
regions_df=NFA_2_df[['country','UN_region','UN_subregion']].drop_duplicates(keep="first").reset_index(drop=True)

In [6]:
population_df=NFA_2_df[["country","year","population"]].drop_duplicates(keep="first").reset_index(drop=True)

In [7]:
records_list_df=NFA_df["record"].drop_duplicates(keep="first")

In [8]:
e_footprint_df=NFA_2_df[["country","year",'record', 'crop_land', 'grazing_land', 'forest_land', 'fishing_ground',
       'built_up_land', 'carbon']].loc[NFA_df["record"].isin(["BiocapTotGHA"," EFConsTotGHA","EFExportsTotGHA","EFImportsTotGHA","EFProdTotGHA"])]

## Use pandas to load csv converted DataFrame into database

In [13]:
rds_connection_string = "root:mohammad@localhost/national_footprint"
engine = create_engine(f'mysql://{rds_connection_string}')

In [16]:
regions_df.to_sql(name ='region',if_exists = 'replace', con = engine, index = False)
population_df.to_sql(name ='population',if_exists = 'replace', con = engine, index = False)
records_list_df.to_sql(name ='records',if_exists = 'replace', con = engine, index = False)
e_footprint_df.to_sql(name ='e_footprint',if_exists = 'replace', con = engine, index = False)
### Confirm data has been added by querying the customer_location table
#pd.read_sql_query('select * from region', con=engine).head()

## countries' land area from wikipedia page

In [ ]:
url="https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area"
country_area_list=pd.read_html(url)
country_area_df=country_area_list[0]
columns=list(country_area_df.loc[0])
print(columns)
columns=['Rank', 'country', 'Total in km2 (mi2)', 'Land in km2 (mi2)', 'Water in km2 (mi2)', '% water', 'Notes']
country_area_df.columns=columns
country_area_df=country_area_df.iloc[2:]
country_area_df=country_area_df.drop(columns="Rank").reset_index(drop=True)
print(country_area_df.shape)
country_area_df.head()

In [ ]:
country_area_df=country_area_df[country_area_df["country"].isin(countries_df["country"])]
country_area_df.shape

In [ ]:
area_df=pd.merge(countries_df, country_area_df,on="country",how="left").sort_values("country")

area_df.shape
area_df[area_df["country"].isin(["Denmark"])]

In [ ]:
trial2=countries_df[(~countries_df["country"].isin(country_area_df["country"]))]["country"]
trial2

In [ ]:
rds_connection_string = "localhost:mohammad@127.0.0.1/co2_emissions"
engine = create_engine(f'mysql://{rds_connection_string}')

## World CO2 emissions from consumption of energy from theguardian.com

In [ ]:
co2 = pd.read_excel('World CO2 emissions from consumption of energy.xlsx',sheet_name= 'Total Carbon Dioxide Emissions')

In [ ]:
co2.columns = co2.iloc[0,]

In [ ]:
co2.reset_index(inplace= True)

In [ ]:
co2 = co2.reindex(co2.index.drop(0))

In [ ]:
co2.rename(columns = {'level_0':'rank_2009','level_1':'rank_2008','Rank, 2006':'rank_2006',
                     'ISO country code': 'iso_code'},inplace = True)
co2.columns.name = None

In [ ]:
clean = co2.loc[~co2['iso_code'].isnull(),]

In [ ]:
# Table!! Create the ISO Code and Country List
iso_country = clean[['iso_code','Country']].drop_duplicates().reset_index(drop = True).reset_index().rename(columns = {'Country':'country','index':'id'})
iso_country.head()

# remove duplicated isocode and year combinations
iso_country = iso_country.loc[~iso_country.duplicated(subset=['iso_code','country'])]

# UM map to both 'Wake Island' and U.S. Pacific Islands. After research, update UM for U.S. Pacific Islands to UM1
#iso_country[iso_country['country'] == 'U.S. Pacific Islands']['iso_code'] = 'UM1'
print(iso_country.head())
#iso_country[iso_country["iso_code"] == "CN"]["country"]="chinaa"


In [ ]:
trial3=pd.merge(iso_country,countries_df,how="outer",on="country").sort_values("country").reset_index(drop=True)
trial4=trial3[trial3.isnull().any(axis=1)]
print(trial4)

In [ ]:
##### rank_list = ['rank_2009','rank_2008','rank_2006','% change,  2008 to 2009','% change 2000 to 2009','Change in place, 2008 to 2009']
clean_rank_change = clean[[*rank_list,'iso_code']].set_index('iso_code')
clean_rank_change.head()

In [ ]:
#remove the percentage changes
clean_rank_new = clean_rank_change[['rank_2009','rank_2008','rank_2006']].rename(columns = lambda x : x.replace('rank_',''))

clean_rank_new = clean_rank_new.stack().reset_index().rename(columns = {'level_1':'year',0:'rank'})


clean_rank_new = clean_rank_new[~clean_rank_new.duplicated(subset = ['iso_code','year'])]
clean_rank_new.head()

In [ ]:
##Table!! Create the Country, Year, and CO2 Emission table
clean_co2 = clean.drop(rank_list,axis = 1)
clean_co2 = clean_co2.set_index('iso_code').drop('Country',axis = 1)
clean_co2 = clean_co2.stack().reset_index().rename(columns = {'level_1':'year',0:'co2_emission'})

# remove duplicated isocode and year combinations
clean_co2 = clean_co2.loc[~clean_co2.duplicated(subset=['iso_code','year'])]